# 나만의 여행 가이드 보이스 챗봇

### 사용자 입력을 음성으로 받는다.

### STT를 적용하여 텍스트로 변환한다.

### 변환된 텍스트를 입력으로 하여 프롬프트 엔지니어링을 해 API 요청을 보낸다. 

### 반환받은 응답을 TTS를 적용하여 음성으로 재생한다. 

### (+) 2에서 입력된 텍스트와 4에서 반환된 응답을 채팅 내역 보듯이 (카카오톡 대화처럼) 화면에 출력한다.

In [20]:
from dotenv import load_dotenv
import os

load_dotenv()
OPEN_API_KEY = os.getenv('OPEN_API_KEY')

In [21]:
import speech_recognition as sr

recognizer = sr.Recognizer()

with sr.Microphone() as source:
    print("말씀하세요...")
    audio = recognizer.listen(source)  
    with open("tts_output_voice.mp3", "wb") as f:
        f.write(audio.get_wav_data())   

말씀하세요...


In [22]:
from openai import OpenAI
client = OpenAI()

with open("tts_output_voice.mp3", "rb") as f:
    transcriptions = client.audio.transcriptions.create(
        model="whisper-1",
        file=f,
        language="ko"
    )

query_text = transcriptions.text
print("음성 인식 결과:", query_text)

음성 인식 결과: 피자의 나라 이탈리아 2박 5일


In [23]:
from openai import OpenAI
import json

def my_travel_guide(query: str, temperature: float = 0.5, model: str = "gpt-4o-mini"):
    """
    '나만의 여행 가이드 보이스 챗봇'
    사용자의 여행 조건(query)에 맞춰 맞춤 여행 코스/팁을 JSON으로 반환
    """
    client = OpenAI()

    system_instruction = """
너는 친절한 '개인 여행 가이드'다. 한국어로 답한다.
사용자가 제공한 여행 조건(도시, 기간, 동행자, 예산, 관심사 등)을 분석하여
실행 가능한 맞춤 여행 일정을 제안해.

[응답 원칙]
- 하루 단위 또는 시간대별 동선으로 추천 코스를 정리.
- 유명 관광지 + 숨은 명소를 균형 있게 제안.
- 각 관광지/활동에 대해 간단한 설명뿐 아니라 추가적인 배경·특징 설명을 제공.
- 교통(대중교통/도보/렌트카)과 티켓 팁(예매, 휴무일) 포함.
- 날씨/혼잡도 주의사항 반영.
- 비/휴무 대비 대체 플랜 반드시 포함.
- 말투는 친근하고 간결하게.

[출력(JSON)]
{
  "trip_summary": "한 줄 요약",
  "itinerary": [
    {
      "day": 1,
      "schedule": [
        {
          "time": "오전",
          "place": "관광지/활동",
          "note": "간단한 설명",
          "description": "관광지의 역사, 특징, 방문 포인트 등 추가 설명"
        },
        {
          "time": "오후",
          "place": "관광지/활동",
          "note": "간단한 설명",
          "description": "관광지의 역사, 특징, 방문 포인트 등 추가 설명"
        }
      ]
    }
  ],
  "food_recommendations": [
    {"name": "맛집/카페", "comment": "추천 이유"}
  ],
  "transport_tips": ["교통/티켓 관련 팁"],
  "weather_notes": ["날씨/혼잡도 주의사항"],
  "backup_plan": ["비/휴무 대비 대체 일정"]
}
"""

    user_message = f"""
[여행 조건/상황]
{query}

조건이 부족하면 도시, 기간, 동행자, 예산, 관심사를 합리적으로 보완해줘.
JSON 스키마의 키는 반드시 그대로 유지한다.
"""

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": user_message},
        ],
        response_format={"type": "json_object"},
        temperature=temperature,
        max_tokens=1500,
    )

    content = resp.choices[0].message.content
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        return {"raw_text": content}


In [24]:
result = my_travel_guide(query_text)

print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "trip_summary": "이탈리아에서의 2박 5일, 로마와 피렌체의 매력을 만끽하세요!",
  "itinerary": [
    {
      "day": 1,
      "schedule": [
        {
          "time": "오전",
          "place": "콜로세움",
          "note": "고대 로마의 상징",
          "description": "콜로세움은 로마의 대표적인 유적지로, 고대 로마에서 검투사 경기 등이 열렸던 곳입니다. 역사적 가치가 크며, 내부 투어를 통해 당시의 분위기를 느낄 수 있습니다."
        },
        {
          "time": "오후",
          "place": "포로 로마노",
          "note": "고대 로마의 정치 중심지",
          "description": "포로 로마노는 로마의 정치적, 사회적 중심지였습니다. 이곳에서 고대 로마의 유적을 탐방하며, 각 건물의 역사적 배경을 배울 수 있습니다."
        }
      ]
    },
    {
      "day": 2,
      "schedule": [
        {
          "time": "오전",
          "place": "바티칸 시국",
          "note": "세계에서 가장 작은 국가",
          "description": "바티칸 시국은 교황의 거주지로, 성 베드로 대성당과 시스티나 성당이 유명합니다. 미켈란젤로의 작품을 감상할 수 있는 기회입니다."
        },
        {
          "time": "오후",
          "place": "트레비 분수",
          "note": "로마의 대표적인 분수",
          "description": "트레비 분수는 로마에서 가장 아름다운 분수 중 하나로, 동전을 던지면 다시 로마에 돌아온다는 전설이 있습니다."


In [25]:
import json
from openai import OpenAI

result = my_travel_guide(query_text)

print(f"[사용자] {query_text}")
print("[가이드] 여행 일정을 정리해드릴게요!\n")

for day in result["itinerary"]:
    print(f"{day['day']}일차 일정")
    for sch in day["schedule"]:
        print(f"{sch['time']} - {sch['place']} ({sch['note']})")
        print(f"{sch['description']}")
    print()


tts_input = json.dumps(result, ensure_ascii=False, indent=2)

client = OpenAI()

with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="nova",
    input=tts_input
) as response:
    response.stream_to_file("guide_reply.mp3")


[사용자] 피자의 나라 이탈리아 2박 5일
[가이드] 여행 일정을 정리해드릴게요!

1일차 일정
오전 - 콜로세움 (고대 로마의 상징)
콜로세움은 로마의 대표적인 유적지로, 고대 검투사 경기장으로 사용되었습니다. 역사적 배경을 이해하고, 내부를 탐방해보세요.
오후 - 포로 로마노 (고대 로마의 중심)
포로 로마노는 로마 제국의 정치, 상업, 종교 중심지로, 다양한 유적을 탐방할 수 있습니다. 주변의 유적지를 함께 둘러보세요.

2일차 일정
오전 - 바티칸 시국 (세계에서 가장 작은 국가)
바티칸 시국에는 성 베드로 대성당과 시스티나 성당이 있습니다. 미켈란젤로의 천장화는 꼭 보세요.
오후 - 피렌체 이동 및 우피치 미술관 (예술의 도시)
기차로 피렌체로 이동 후, 우피치 미술관에서 보티첼리, 다빈치의 작품을 감상하세요. 미술관은 사전 예약 필수입니다.

3일차 일정
오전 - 두오모 성당 (피렌체의 상징)
두오모 성당은 독특한 돔 구조로 유명하며, 전망대에서 피렌체 시내를 한눈에 볼 수 있습니다.
오후 - 피렌체 시내 탐방 (로컬 맛집 탐방)
피렌체의 좁은 골목길을 걸으며 다양한 가게와 카페를 탐방하세요. 현지인들이 추천하는 맛집을 찾아보세요.



In [ ]:
import os, json, glob
from openai import OpenAI

HISTORY_FILE = "chat_history_mp3.json"
client = OpenAI()

if os.path.exists(HISTORY_FILE):
    with open(HISTORY_FILE, "r", encoding="utf-8") as f:
        chat_history = json.load(f)
else:
    chat_history = [] 

result = my_travel_guide(query_text)

chat_history.append(f"[사용자] {query_text}")

reply_lines = ["여행 일정을 정리해드릴게요!"]
for day in result.get("itinerary", []):
    reply_lines.append(f"{day['day']}일차 일정")
    for sch in day.get("schedule", []):
        reply_lines.append(f"{sch.get('time','')} - {sch.get('place','')} ({sch.get('note','')})")
        if sch.get("description"):
            reply_lines.append(sch["description"])
reply_text = "\n".join(reply_lines).strip()

next_idx = len(glob.glob("guide_reply_*.mp3")) + 1
mp3_name = f"guide_reply_{next_idx:03}.mp3"

with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="nova",
    input=reply_text
) as response:
    response.stream_to_file(mp3_name)

chat_history.append(f"[가이드 🎧] {mp3_name}")

for i in range(0, len(chat_history), 2):
    print("-" * 60)
    print(chat_history[i])           
    if i + 1 < len(chat_history):
        print(chat_history[i + 1])   
    print("-" * 60)

with open(HISTORY_FILE, "w", encoding="utf-8") as f:
    json.dump(chat_history, f, ensure_ascii=False, indent=2)

------------------------------------------------------------
[사용자] 중국 2박 2일
[가이드 🎧] guide_reply_002.mp3
------------------------------------------------------------
------------------------------------------------------------
[사용자] 피자의 나라 이탈리아 2박 5일
[가이드 🎧] guide_reply_003.mp3
------------------------------------------------------------
